# Arbol de Decisión
Un árbol de decisión es un modelo de aprendizaje automático que toma decisiones basadas en una serie de preguntas (nodos) para clasificar o predecir datos. Cada pregunta divide los datos en ramas, lo que lleva a decisiones finales (hojas) que representan las predicciones. Es una técnica de aprendizaje supervisado utilizada en clasificación y regresión.
Exportamos el DataFrame ya trabajado anteriormente.

In [18]:
import pandas as pd
import matplotlib.pyplot as plt

trata_exp = pd.read_csv('../trata_exp.csv') 
trata_exp.shape

(5741, 13)

### Adecuaciones para trabajar en el modelo

#### Filtrado de Sexo y Edad

Se eliminan las filas del Sexo "Desconocido" (Unknown) y "Otros" (Others) que representan el 10.9% del total.

In [19]:
df = trata_exp.copy()
df = df[df['Sex'] != 'Unknown']
df = df[df['Sex'] != 'Other']
df = df[df['Age'] != 'Unknown']
df.shape

(5118, 13)

### Valores extremos (outliner)
Se procede a eliminar los valors extremos.

In [6]:
# Calcula el rango intercuartil (IQR)
Q1 = df['Value'].quantile(0.25)
Q3 = df['Value'].quantile(0.75)
IQR = Q3 - Q1

# Define los límites para identificar valores atípicos 2 veces el rango intercuartil
limite_inferior = Q1 - 2 * IQR
limite_superior = Q3 + 2 * IQR

# Crea una máscara booleana para identificar las filas con valores atípicos.
mascara_atipicos = (df['Value'] < limite_inferior) | (df['Value'] > limite_superior)

# Filtra el DataFrame para eliminar las filas con valores atípicos
df_sin_atipicos = df[~mascara_atipicos]
# Se crea una copia para evitar dataframes rebanados (slices)
df_arbol = df_sin_atipicos.copy()


Se crean las etiquetas para las variables categóricas: Categoría, Sexo y Edad.

In [7]:
from sklearn.preprocessing import LabelEncoder

# Crear una instancia de LabelEncoder para cada variable categórica
label_encoder_category = LabelEncoder()
label_encoder_sex = LabelEncoder()
label_encoder_age = LabelEncoder()

# Aplicar LabelEncoder a cada variable categórica
df_arbol['Category_encoded'] = label_encoder_category.fit_transform(df_arbol['Category'])
df_arbol['Sex_encoded'] = label_encoder_sex.fit_transform(df_arbol['Sex'])
df_arbol['Age_encoded'] = label_encoder_age.fit_transform(df_arbol['Age'])

nuevo_orden = ['Category_encoded','Age_encoded', 'Sex_encoded' ]
df_arbol = df_arbol[nuevo_orden]
df_arbol.sample(15)

,Category_encoded,Age_encoded,Sex_encoded
532,2,0,1
1554,0,0,1
185,0,1,1
699,2,0,0
5,2,1,0
4709,1,0,0
1367,0,0,1
3983,1,0,1
1509,2,0,1
1488,2,0,0


Las codificación de las categorías son: *CATEGORÍA: 0 EXP. POR FUERZA LABORAL, 1 EXP. SEXUAL Y 2 OTRAS FORMAS DE EXPLOTACIÓN, - EDAD: 0 MENOR DE 18 AÑOS, 1 MAYOR DE 18 AÑOS, - SEXO: 0 FEMENINO, 1 MASCULINO*

## Entrenamiento del modelo.
Se intenta explicar la categoría del delito (variable y) por las categorias sexo y edad (variables x).
Se entrena bajo el criterio de Entropía y se imprime el número de precisión.

In [13]:
from sklearn.tree import DecisionTreeClassifier
y = df_arbol.iloc[:,0]
x = df_arbol.iloc[:,1:]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


arbol = DecisionTreeClassifier(criterion='entropy', max_depth=5)
arbol.fit(x_train, y_train)
prediccion = arbol.predict(x_test)

from sklearn.metrics import accuracy_score
print(f'La precisión del modelo es: {accuracy_score(y_test, prediccion)}')
from sklearn import tree
print(tree.export_text(arbol, feature_names=['Age', 'Sex'], class_names=['Forced labour', 'Sexual exploitation', 'Other forms of exploitation']))


La precisión del modelo es: 0.3429228998849252
|--- Sex <= 0.50
|   |--- Age <= 0.50
|   |   |--- class: Forced labour
|   |--- Age >  0.50
|   |   |--- class: Sexual exploitation
|--- Sex >  0.50
|   |--- Age <= 0.50
|   |   |--- class: Forced labour
|   |--- Age >  0.50
|   |   |--- class: Other forms of exploitation



Se verifica la cantidad de clases: 3, que son las categorias de violencia.

In [9]:
num_clases = len(arbol.classes_)
print(f'Número de clases: {num_clases}')

Número de clases: 3


Se importa el gráfico del árbol en PDF (arbol.pdf).

In [15]:
from sklearn.tree import export_graphviz
import graphviz

dot_data = export_graphviz(arbol, out_file=None, feature_names=['Age', 'Sex'], class_names=['Forced labour', 'Sexual exploitation', 'Other forms of exploitation'], filled=True, rounded=True, special_characters=True)
graph = graphviz.Source(dot_data)
graph.render("arbol")  # Esto creará un archivo "arbol.pdf" en el directorio actual
graph.view("arbol")    # Esto abrirá la representación gráfica del árbol en un visor PDF

'arbol.pdf'

Se prueba bajo el criterio del coeficiente de Gini si es posible mejorar la predición. De todas formas la precisión es similar.

In [14]:
from sklearn.tree import DecisionTreeClassifier
y = df_arbol.iloc[:,0]
x = df_arbol.iloc[:,1:]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


arbol_gini = DecisionTreeClassifier(criterion='gini', max_depth=5)
arbol_gini.fit(x_train, y_train)
prediccion = arbol_gini.predict(x_test)

from sklearn.metrics import accuracy_score
print(f'La precisión del modelo es: {accuracy_score(y_test, prediccion)}')
from sklearn import tree
print(tree.export_text(arbol_gini, feature_names=['Age', 'Sex'], class_names=['Forced labour', 'Sexual exploitation', 'Other forms of exploitation']))

La precisión del modelo es: 0.3233601841196778
|--- Sex <= 0.50
|   |--- Age <= 0.50
|   |   |--- class: Forced labour
|   |--- Age >  0.50
|   |   |--- class: Sexual exploitation
|--- Sex >  0.50
|   |--- Age <= 0.50
|   |   |--- class: Other forms of exploitation
|   |--- Age >  0.50
|   |   |--- class: Other forms of exploitation



Se imprime el gráfico de arbol (arbol2.pdf).

In [16]:

from sklearn.tree import export_graphviz
import graphviz

dot_data = export_graphviz(arbol_gini, out_file=None, feature_names=['Age', 'Sex'], class_names=['Forced labour', 'Sexual exploitation', 'Other forms of exploitation'], filled=True, rounded=True, special_characters=True)
graph = graphviz.Source(dot_data)
graph.render("arbol2")  # Esto creará un archivo "arbol.pdf" en el directorio actual
graph.view("arbol2")    # Esto abrirá la representación gráfica del árbol en un visor PDF

'arbol2.pdf'

#### Conclusión Breve
Del árbol de decisión se puede clasificar que las mujeres menores corren más riesgo de sufrir explotación laboral, en el caso de mayores de edad explotación sexual (entropía). Igual situación para lo analizado bajo Gini. En el caso de los varones menores se clasifica como explotación laboral y para mayores en otras formas de explotación por entropía. Igual consideración para Gini.
En ambos casos la precisión no es la esperada por lo que deberá trabajarse con los obtenidos.